# Import des modules

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from ast import literal_eval
import csv

# Création des variables

In [2]:
DF0 = pd.read_csv("./../Tables/ReferentielPrenoms.csv",converters={"FormesNonNormalisées": literal_eval}).set_index("PrénomNormalisé")
DF0

,PrénomFrançais,Types,FormesNonNormalisées
PrénomNormalisé,,,
Adalhelm,Alleaume,"['masc', 'germanique']","[Adalhelm, Adelhelm, Alhelm]"
Adam,Adam,"['masc', 'hébraïque']",[Adam]
Adolff,Adolphe,"['masc', 'germanique']","[Adolf, Adolff]"
Andres,André,"['masc', 'grec']",[Andres]
Anshelm,Anselme,"['masc', 'germanique']",[Anshelm]
...,...,...,...
Wetzel,NaN,"['masc', 'germanique']",[Wetzel]
Wilhelm,Guillaume,"['masc', 'germanique']",[Wilhelm]
Wirich,NaN,"['masc', 'germanique']","[Wirich, Wyrich]"


# Création de la soupe

In [3]:
soup = BeautifulSoup(open('./../../../Documents/base stras/aves_aa_195_p1.xml', 'r'))

# Création de la liste des éléments de la soupe à comparer

In [4]:
ListePrénomDoc = []
for forename in soup.find_all('forename'):
    ListePrénomDoc.append(forename.get_text().title())
print (ListePrénomDoc)

['Johanns', 'Niclas', 'Hanns', 'Heinrich', 'Walther', 'Bernhart', 'Hanns', 'Hannsheinrich', 'Friedrich', 'Hans Hiltebrant', 'Hannsheinrich', 'Hanns', 'Cüne', 'Jerge', 'Hanns', 'Burckart', 'Hanns', 'Jacob', 'Bernhart', 'Hesseman', 'Claus', 'Hanns', 'Niclaus', 'Hanns Cünrat', 'Cünrat', 'Ulrich', 'Claus Friderich', 'Wolffhelm', 'Walther', 'Bernhart', 'Hanns', 'Peter', 'Reimbolt', 'Reimbolt', 'Jerge', 'Hanns', 'Burckart', 'Diebolt', 'Claus', 'Ludwig', 'Reimbolt', 'Hanns Balthazar', 'Hanns', 'Hanns', 'Claus', 'Thoman', 'Thoman', 'Cüntze', 'Jacob', 'Hanns Tietrich', 'Hanns', 'Boldel', 'Peter', 'Caspar', 'Hanns', 'Hanns', 'Diebolt', 'Thoman', 'Bechtolt', 'Hanns', 'Claus', 'Bechtolt', 'Hanns', 'Hanns', 'Hanns', 'Erbe', 'Obreht', 'Hanns', 'Cunrat', 'Hanns', 'Steffan', 'Hanns', 'Reimbolt', 'Hanns', 'Heinrich', 'Heinrich', 'Heinrich', 'Frantz', 'Cüntzelin', 'Heintzman', 'Jacob', 'Caspar', 'Peter', 'Hanns Jerge', 'Dietrich', 'Hanns', 'Hanns', 'Hanns', 'Peter', 'Volmar', 'Contz', 'Henß', 'Hanns', '

# Comparaison de la liste de la soupe à notre dataframe

In [26]:
for element in ListePrénomDoc :
    
    #Création des listes de références à partir du dataframe DF1.
    Colonne_FormesNonNormalisées = DF0['FormesNonNormalisées'].tolist()
    Colonne_PrénomNormalisé = DF0.index.tolist()
    
    #Vérification de l'existence de la forme non normalisée dans la liste des listes de formes non-normalisées.
    CheckForenameNN = any(element in sublist for sublist in Colonne_FormesNonNormalisées)
    DicoPersonne = {}

    #Sinon, on demande un str en entrée pour le comparer à la liiste des formes normalisées.
    if CheckForenameNN != True :
        ForenameN = input('Entrez la forme normalisée du prénom ' + element + ' :')
        
        #Si l'entrée figure dans la liste des prénoms normalisées, on ajoute la forme non normalisée à la liste de celles-ci dans le dataframe.
        if ForenameN in Colonne_PrénomNormalisé:
            CelluleModif = DF0.loc[ForenameN, 'FormesNonNormalisées']
            #print(type(CelluleModif))
            #print(CelluleModif)
            CelluleModif.append(element)
            
        #Sinon, on crée un dataframe qu'on additionnera à l'existant.
        else :
            
            #On crée le couple 'PrénomNormalisé' dont la valeur correspond à la forme la plus courante du prénom.
            DicoPersonne['PrénomNormalisé'] = ForenameN
            
            #On crée le couple 'FormeFR' dont la valeur correspond à la traduction française du prénom.
            FormeFR = input('Entrez la traduction française du prénom ' + element +' (possibilité de laisser vide)')
            DicoPersonne['PrénomFrançais'] = FormeFR
            
            #On crée le couple 'Types' dont la valeur est une liste contenant des tags proposant une description typologique du prénom.
            typologie = []
            n = int(input('Entrez le nombre de types à attribuer au prénom ' + element + ' :'))
            for i in range(0,n):
                Type = input('Entrez un des types à attribuer au prénom ' + element + ' :')
                typologie.append(Type)
            DicoPersonne['Types'] = typologie
            
            #On crée le couple 'FormesNonNormailglisées' dont la valeur est une liste contenant toutes les formes alternatives du prénom.
            ListeNN = []
            ListeNN.append(element)
            if element != ForenameN :
                ListeNN.append(ForenameN)
            DicoPersonne['FormesNonNormalisées'] = ListeNN
            
            #On crée le dataframe à ajouter à l'existant.
            DF1 = pd.DataFrame([DicoPersonne]).set_index('PrénomNormalisé')
            
            #On concatène le dictionnaire obtenu à notre Referentiel avant de recommencer toute l'opération.
            DF0 = pd.concat([DF0,DF1])
DF0

Entrez la forme normalisée du prénom Fredel :Friedrich
Entrez la forme normalisée du prénom Anßhelm :Anshelm
Entrez la forme normalisée du prénom Gylge :Gilge
Entrez la forme normalisée du prénom Mydehart :Midhart
Entrez la traduction française du prénom Mydehart (possibilité de laisser vide)Médard
Entrez le nombre de types à attribuer au prénom Mydehart :2
Entrez un des types à attribuer au prénom Mydehart :masc
Entrez un des types à attribuer au prénom Mydehart :germanique
Entrez la forme normalisée du prénom Struße :Strauss
Entrez la traduction française du prénom Struße (possibilité de laisser vide)
Entrez le nombre de types à attribuer au prénom Struße :2
Entrez un des types à attribuer au prénom Struße :masc
Entrez un des types à attribuer au prénom Struße :germanique
Entrez la forme normalisée du prénom Josel :Jost
Entrez la forme normalisée du prénom Ulin :Uhlmann
Entrez la forme normalisée du prénom Amandus :Amandus
Entrez la traduction française du prénom Amandus (possibilité

,PrénomFrançais,Types,FormesNonNormalisées
PrénomNormalisé,,,
Adalhelm,Alleaume,"['masc', 'germanique']","[Adalhelm, Adelhelm, Alhelm]"
Adam,Adam,"['masc', 'hébraïque']",[Adam]
Adolff,Adolphe,"['masc', 'germanique']","[Adolf, Adolff]"
Andres,André,"['masc', 'grec']",[Andres]
Anshelm,Anselme,"['masc', 'germanique']","[Anshelm, Anßhelm]"
...,...,...,...
Midhart,Médard,"[masc, germanique]","[Mydehart, Midhart]"
Strauss,,"[masc, germanique]","[Struße, Strauss]"
Amandus,Amand,"[masc, latin]",[Amandus]


In [29]:
DF0.sort_values('PrénomNormalisé')
DF0.to_csv("./../Tables/ReferentielPrenoms.csv")